In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import os
import cv2
import torch
import re
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

# === CONFIGURATION ===
image_dir = "/content"
prompt = "a red rose gently waving in the wind"
model_id = "sd-legacy/stable-diffusion-v1-5"
num_images = 50
strength = 0.7
frames_between = 30
fps = 15
video_filename = "rose_video.mp4"

# === STEP 1: GENERATE IMAGES IF NOT PRESENT ===
rose_images = sorted([
    f for f in os.listdir(image_dir)
    if re.match(r'rose\d+\.png', f)
], key=lambda x: int(re.search(r'\d+', x).group()))

if len(rose_images) < num_images:
    print(f"Generating {num_images} rose images...")
    pipe_txt2img = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
    pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

    image = pipe_txt2img(prompt).images[0]
    image.save(os.path.join(image_dir, "rose1.png"))

    for i in range(2, num_images + 1):
        image = pipe_img2img(prompt=prompt, image=image, strength=strength).images[0]
        image.save(os.path.join(image_dir, f"rose{i}.png"))

    rose_images = [f"rose{i}.png" for i in range(1, num_images + 1)]

# === STEP 2: LOAD AND VALIDATE IMAGES ===
image_paths = [os.path.join(image_dir, f) for f in rose_images]
images = [cv2.imread(path) for path in image_paths]

if any(img is None for img in images):
    raise RuntimeError("One or more images could not be loaded.")

# === STEP 3: RESIZE IMAGES ===
target_size = (images[0].shape[1], images[0].shape[0])  # (width, height)
resized_images = [cv2.resize(img, target_size, interpolation=cv2.INTER_LINEAR) for img in images]

# === STEP 4: INTERPOLATE FRAMES ===
interpolated_frames = []
for i in range(len(resized_images) - 1):
    for j in range(frames_between):
        alpha = j / frames_between
        frame = cv2.addWeighted(resized_images[i], 1 - alpha, resized_images[i + 1], alpha, 0)
        interpolated_frames.append(frame)
interpolated_frames.append(resized_images[-1])  # Add final image

# === STEP 5: CREATE VIDEO ===
frame_height, frame_width = target_size[1], target_size[0]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))

for frame in interpolated_frames:
    video_writer.write(frame)

video_writer.release()
print(f"✅ Video saved as {video_filename}")


Generating 50 rose images...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

...

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

✅ Video saved as rose_video.mp4
